In [30]:
import os
import csv
from pybedtools import BedTool
import numpy as np
import pandas as pd

In [31]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

# raise StopExecution

collect all files

In [32]:
path = "/private8/Projects/itamar/ZNF/Sra_GSE73211_35_samples/REDI/curated/nodups_cov5_HE_PLUS_STAR100len/Edits_only/"
all_samples = []
for outf in [path+f for f in os.listdir(path) if f.endswith('.out.tab')]:
    with open(outf,'r') as ffd:
        DictReader_obj = csv.DictReader(ffd, delimiter="\t")
        # content = ffd.readlines()
        file_as_list_dict = [line for line in DictReader_obj]
        all_samples.append(file_as_list_dict)


filter snps

In [33]:

#filter 100% editing sites
all_files_sites_no_snp_raw =[]
for sample in all_samples:
        # temp = [site for site in sample if 0.001 < float(site["Frequency"]) < 0.98] 
        # TODO also filte 45 -55%?
        temp  = [site for site in sample if ((0.001 < float(site["Frequency"]) < 0.98) and  (0.55 < float(site["Frequency"]) or float(site["Frequency"]) < 0.45))]
        all_files_sites_no_snp_raw.append(temp.copy())

#fiter snps from dbsnp
temp_F_path = path+"temp_pos.bed"
sites_as_bed =  [site["Region"]+"\t"+str(int(site["Position"])-1)+"\t"+site["Position"]+"\n" for sample in all_files_sites_no_snp_raw for site in sample]
uniq = set(sites_as_bed)
#print(len(uniq))
with open(temp_F_path, "w") as outFD:
    outFD.writelines(uniq)
# intersect with snps
snps = BedTool("/private/common/Data/dbSNP-153/hg38-dbSNP153-common.bed")
sites = BedTool(temp_F_path)
result_path = path+"temp_result.bed"
# write and read results
(sites-snps).saveas(result_path)
with open(result_path, "r") as resFD:
    as_bed_no_snp = resFD.readlines()
#print(len(as_bed_no_snp))
# remove temp file
os.remove(result_path)
os.remove(temp_F_path)
# save the result as positions
no_snp_positions = [(pos.split(sep="\t")[0], pos.split(sep="\t")[2][:-1])for pos in as_bed_no_snp]
all_files_sites_no_snp = []
# save for every sample the passed sites
for sample in all_files_sites_no_snp_raw:
    all_files_sites_no_snp.append([site for site in sample if (site["Region"], site["Position"]) in no_snp_positions])
# save list of all reults (contains duplicats!)
all_sites_list_no_snp = [site for sample in all_files_sites_no_snp for site in sample]


create table of all sites composing data for every uniq site

In [44]:
# create uniq list of all sites
uniq_sites = []
sites_dict = dict()
already = set()
for site_org in all_sites_list_no_snp:
    site=site_org.copy()
    # make set from subs
    site["AllSubs"] = set(site["AllSubs"].split(" "))
    site["BaseCount[A,C,G,T]"]  = np.array([int(x) for x in site["BaseCount[A,C,G,T]"].replace("[","").replace("]","").replace(" ","").split(",")])
    site['Coverage-q25'] = float(site['Coverage-q25'])
    site['MeanQ'] = float(site['MeanQ'])
    site['Frequency'] = float(site['Frequency'])
    siteID = site["Region"]+"_"+site["Position"]
    if siteID not in sites_dict:
        sites_dict[siteID] = site.copy()
    else:
        sites_dict[siteID]["AllSubs"].update(site["AllSubs"].copy())
        sites_dict[siteID]["BaseCount[A,C,G,T]"] += site["BaseCount[A,C,G,T]"]
        total_cov = sites_dict[siteID]['Coverage-q25'] + site['Coverage-q25']
        sites_dict[siteID]['MeanQ'] = round(site['MeanQ']*(site['Coverage-q25']/total_cov) + sites_dict[siteID]['MeanQ']*(sites_dict[siteID]['Coverage-q25']/total_cov),1)
        sites_dict[siteID]['Frequency']= round(site['Frequency']*(site['Coverage-q25']/total_cov) + sites_dict[siteID]['Frequency']*(sites_dict[siteID]['Coverage-q25']/total_cov),4)
        sites_dict[siteID]['Coverage-q25'] += site['Coverage-q25']
    
# after collecting all sites ->calculations
for site in sites_dict.values():
    # calc frequncy again
    b_c=site["BaseCount[A,C,G,T]"].copy()
    n_options = ['A','C','G','T']
    #find the original nucleotide without editing
    orig_pos=np.argmax(b_c)
    orig_nuc=n_options[orig_pos]
    b_c[orig_pos]=0
    # find the edited to
    new_pos=np.argmax(b_c)
    new_nuc=n_options[new_pos]
    # save mismatch type and Frequency
    site['MM']=orig_nuc+new_nuc
    site['CountFrequency']=round(site["BaseCount[A,C,G,T]"][new_pos]/np.sum(site["BaseCount[A,C,G,T]"]),4)
    # caculate AG frequncy by base count
    site['CountAGFrequency']=round(site["BaseCount[A,C,G,T]"][2]/np.sum(site["BaseCount[A,C,G,T]"]),4)
    # convert BaseCount to string
    site["BaseCount[A,C,G,T]"] = ";".join([str(x) for x in site["BaseCount[A,C,G,T]"].tolist()])
    site["AllSubs"] = ";".join(site["AllSubs"])
# convert dict to list
uniq_sites = list(sites_dict.values())
print("total: ",len(all_sites_list_no_snp),"\n uniq: ", len(uniq_sites))



total:  137393 
 uniq:  65300


In [ ]:
# for site in all_sites_list_no_snp:
#     if site['Position'] == "54375862":
#         print(site)
    

filter by Frequency

In [70]:
# filter by cov > 100 and some MM's freq > 0.01
uniq_sites_filtered = [site for site in uniq_sites if (site["CountFrequency"] >= 0.01 and site['Coverage-q25'] > 100)]
AG_uniq_sites_filtered = [site for site in uniq_sites if (site["CountAGFrequency"] >= 0.01 and site['Coverage-q25'] > 100)] 
print("total: ",len(all_sites_list_no_snp),"\n uniq: ", len(uniq_sites_filtered))

total:  137393 
 uniq:  9769


In [73]:
# count number of each MM
df = pd.DataFrame.from_records(uniq_sites_filtered)
print(df['MM'].value_counts())
print(df['MM'].value_counts(normalize=True))


AG    5551
AT    2339
AC    1879
Name: MM, dtype: int64
AG    0.568226
AT    0.239431
AC    0.192343
Name: MM, dtype: float64


save result

In [76]:
#save file
df.to_csv(path_or_buf=path+"new"+"all_uniq_by_frequncy_sites.known.tab",index=False,sep='\t')
print(df.shape[0])
# create and save AG sites only
AG_df=pd.DataFrame.from_records(AG_uniq_sites_filtered)
AG_df.to_csv(path_or_buf=path+"new"+"all_uniq_by_frequncy_sites_A2G.known.tab",index=False,sep='\t')
print(AG_df.shape[0])


# df_A2G_1=df[df['MM']=="AG"]
# print(df_A2G_1.shape[0])
# df_A2G=df[df['AllSubs'].str.contains("AG",regex=False)]
# print(df_A2G.shape[0])
# df.drop(['AllSubs'],axis=1,inplace=True)




9769
5649
